# Transfer Learning for Image Classification

This notebook uses a classifier model that was originally trained using [ImageNet](https://image-net.org) and does transfer learning with either a TF dataset or your own raw images.
The notebook performs the following steps:
1. [Install dependencies and setup parameters](#1.-Install-dependencies-and-setup-parameters)
2. [Prepare the dataset](#2.-Prepare-the-dataset) using either a TF dataset or your own images
3. [Predict using the original model](#3.-Predict-using-the-original-model)
4. [Transfer learning](#4.-Transfer-Learning)
5. [Evaluate the model](#5.-Evaluate-the-model)
6. [Export the saved model](#6.-Export-the-saved-model)

## 1. Install dependencies and setup parameters

In [ ]:
!pip install ipywidgets==7.6.5 \
             tensorflow_hub==0.12.0 \
             tensorflow-datasets==4.4.0 \
             scipy \
             'pandas>=1.1.5' \
             'matplotlib>=3.3.4'

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from model_util import tfhub_model_map

print('Supported models:')
print('\n'.join(tfhub_model_map.keys()))

# Specify the the parent directory for the custom or tf dataset
dataset_directory = os.environ["DATASET_DIR"] if "DATASET_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "dataset")
    
# Specify a directory for output
output_directory = os.environ["OUTPUT_DIR"] if "OUTPUT_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "output")

# Batch size
batch_size = 32

In [ ]:
# Specify a model from the list above
model_name = "efficientnet_b0"

if model_name not in tfhub_model_map.keys():
    raise ValueError("The specified model_name ({}) is invalid. Please select from: {}".
                     format(model_name, tfhub_model_map.keys()))
    
# Get the info for the specified model from the map
model_map_values = tfhub_model_map[model_name]
model_handle = tfhub_model_map[model_name]["imagenet_model"]
feature_vector_handle = tfhub_model_map[model_name]["feature_vector"]
image_size = tfhub_model_map[model_name]["image_size"]
print("Model:", model_name)
print("Classifier model:", model_handle)
print("Feature vector:", feature_vector_handle)
print("Image size:", image_size)

## 2. Prepare the dataset

### Option A: Use a TF dataset

To use a TF Dataset, load from the tensorflow_datasets library, preprocess the images to convert them to float32, and resize the images. This example uses the [Food-101 dataset using the TensorFlow datasets API](https://www.tensorflow.org/datasets/catalog/food101) dataset, but you can choose from a wide variety of [options](https://www.tensorflow.org/datasets/catalog/overview) (click on the "Image classification" section). If the dataset is not found in the dataset directory it is downloaded. Subsequent runs will reuse the already downloaded dataset.

In [ ]:
# Options include: "food101", cats_vs_dogs", "horses_or_humans", "rock_paper_scissors", and "tf_flowers"
tf_dataset = "food101"

# Load the dataset using the TensorFlow datasets API
[train_ds, test_ds], info = tfds.load(tf_dataset,
                       data_dir=dataset_directory,
                       split=["train[:75%]", "train[:25%]"],
                       as_supervised=True,
                       shuffle_files=True,
                       with_info=True)

# Preprocess the images to convert them to float32 and resize the images to match our model
def preprocess_image(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_with_pad(image, image_size, image_size)
    return (image, label)

train_ds = train_ds.map(preprocess_image)
test_ds = test_ds.map(preprocess_image)

print("Dataset directory: ", dataset_directory)
print("Training dataset size:", len(train_ds))
print("Validation dataset size:", len(test_ds))

# Training data is shuffled for randomness
# https://www.tensorflow.org/datasets/keras_example#build_a_training_pipeline
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(info.splits['train'].num_examples)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

# Test data does not need to be shuffled, and caching is done after batching
# https://www.tensorflow.org/datasets/keras_example#build_an_evaluation_pipeline
test_ds = test_ds.batch(batch_size)
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

# Get class names for the dataset
class_names = info.features["label"].names
print("Number of classes:", len(class_names))
print('After processing and batching: ', train_ds.element_spec)

Skip to the next step [3. Predict using the original model](#3.-Predict-using-the-original-model) to continue using the TF Dataset.

### Option B: Use your own image dataset

To use your own image dataset for transfer learning with the rest of this notebook, format your images as `.jpg` files and save them in folders named after the classes that you want the model to predict. To provide a working example using the correct layout, we will download and extract a flower species dataset. This is different from using the TF dataset called `tf_flowers`, although they are the same images, because the download contains image files, not tf_records, and we are not using the TF datasets API. After downloading and extracting, you will have the following  subdirectories in your dataset directory. Each species subfolder will contain numerous `.jpg` files:

```
dataset_directory
└── flower_photos
    └── daisy
    └── dandelion
    └── roses
    └── sunflowers
    └── tulips
```

Use this as an example to organize your own image files accordingly.

In [ ]:
# When you have your own properly organized subdirectory of images, adjust this variable
dataset_subdir = os.path.join(dataset_directory, "flower_photos")

In [ ]:
# Only run this if you want to use the example flowers dataset
if not os.path.exists(dataset_subdir):
    os.mkdir(dataset_subdir)
    !apt-get update && apt-get -qq install curl
    !curl https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz -o flower_photos.tgz
    !tar xvf flower_photos.tgz --directory $dataset_directory

In [ ]:
import pathlib
data_subdir_path = pathlib.Path(dataset_subdir)
image_count = len(list(data_subdir_path.glob('*/*.jpg')))
print('Images:', image_count)
print('Image Size:', image_size)

In [ ]:
# Create an image data generator and partition the data into train and test sets
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.25)
train_ds = img_gen.flow_from_directory(data_subdir_path, batch_size=batch_size, 
                                   target_size=(image_size, image_size),
                                   class_mode='sparse', subset='training')
test_ds = img_gen.flow_from_directory(data_subdir_path, batch_size=batch_size, 
                                  target_size=(image_size, image_size),
                                  class_mode='sparse', subset='validation')

# Get class names for the dataset
class_names = [k for k in train_ds.class_indices.keys()]
print("Number of classes:", len(class_names))

## 3. Predict using the original model

Use the classifier model that was trained using ImageNet to do predictions with the dataset and view the results for a single batch. Table below compares the prediction made by the ImageNet trained model with the actual label from the dataset.

In [ ]:
# Get a batch of the dataset to use for testing
batch = next(iter(test_ds))
image_batch, label_batch = batch

# List of the actual labels for this batch
actual_label_batch = [class_names[int(id)] for id in label_batch]

# Download the ImageNet labels and load them into a list
labels_file = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"
downloaded_file = tf.keras.utils.get_file("labels.txt", origin=labels_file)
imagenet_classes = []

with open(downloaded_file) as f:
    imagenet_labels = f.readlines()
    imagenet_classes = [l.strip() for l in imagenet_labels]

# Predict using the TF Hub classifier that was trained using ImageNet
classifier = tf.keras.Sequential([
    hub.KerasLayer(model_handle, input_shape=(image_size, image_size)+(3,))
])
predicted_batch = classifier.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = [imagenet_classes[id] for id in predicted_id]

# Create a results table to list out the ImageNet class prediction vs the actual dataset label
results_table = []
for prediction, actual in zip(predicted_label_batch, actual_label_batch):
    results_table.append([prediction, actual])

pd.DataFrame(results_table, columns=["ImageNet Prediction", "Actual Label"])

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
    plt.subplot(6,5,n+1)
    plt.imshow(image_batch[n])
    plt.title(predicted_label_batch[n].title(), fontsize=9)
    plt.axis('off')
_ = plt.suptitle("ImageNet predictions")
plt.show()

## 4. Transfer Learning

Get the feature vector from TF Hub and add on a dense layer based on the number of classes in our dataset. Train the model using the training dataset for the specified number of epochs.

In [ ]:
# Number of training epochs
training_epochs = 1

# To reduce training time, the feature extractor layer can remain frozen (do_fine_tuning=False).
# Fine-tuning can be enabled to potentially get better accuracy. Note that enabling fine-tuning
# will increase training time.
do_fine_tuning = False

# Optionally add a dropout layer (set to a float between 0 and 1, or None).
# If set to None, no dropout layer will be added.
dropout_layer_rate = None

In [ ]:
feature_extractor_layer = hub.KerasLayer(feature_vector_handle,
                                         input_shape=(image_size, image_size, 3),
                                         trainable=do_fine_tuning)

if dropout_layer_rate == None:
    model = tf.keras.Sequential([
      feature_extractor_layer,
      tf.keras.layers.Dense(len(class_names))
    ])
else:
    model = tf.keras.Sequential([
      feature_extractor_layer,
      tf.keras.layers.Dropout(dropout_layer_rate),
      tf.keras.layers.Dense(len(class_names))
    ])

model.summary()

In [ ]:
%%time
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

batch_stats_callback = CollectBatchStats()

history = model.fit(train_ds, epochs=training_epochs, shuffle=True, callbacks=[batch_stats_callback])

## 5. Evaluate the model

After the training completes, evaluate the model's accuracy using the validation dataset.

In [ ]:
%%time
model.evaluate(test_ds, batch_size=batch_size)

Also, predict using the same sample batch that we used earlier with the ImageNet trained classier to visualize the results after training the model.

In [ ]:
# Predict using the sample batch
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = [class_names[id] for id in predicted_id]

# Display the results
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
    plt.subplot(6,5,n+1)
    plt.imshow(image_batch[n])
    correct_prediction = actual_label_batch[n] == predicted_label_batch[n]
    color = "darkgreen" if correct_prediction else "crimson"
    title = predicted_label_batch[n].title() if correct_prediction else "{}\n({})".format(predicted_label_batch[n], actual_label_batch[n]) 
    plt.title(title, fontsize=9, color=color)
    plt.axis('off')
_ = plt.suptitle("Model predictions")
plt.show()
print("Correct predictions are shown in green")
print("Incorrect predictions are shown in red with the actual label in parenthesis")

## 6. Export the saved model

In [ ]:
saved_model_dir = os.path.join(output_directory, "{}_saved_model".format(model_name))
model.save(saved_model_dir)

## Dataset citations
```
@inproceedings{bossard14,
  title = {Food-101 -- Mining Discriminative Components with Random Forests},
  author = {Bossard, Lukas and Guillaumin, Matthieu and Van Gool, Luc},
  booktitle = {European Conference on Computer Vision},
  year = {2014}
}

@ONLINE {tfflowers,
author = "The TensorFlow Team",
title = "Flowers",
month = "jan",
year = "2019",
url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }
```